In [2]:
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import csv
from sklearn.model_selection import GridSearchCV

In [2]:
file_paths_GLCM = [
    r'D:\AIL301m\Feature extract\GLCM\Merged_BC-15_GLCM_features.csv',
    r'D:\AIL301m\Feature extract\GLCM\Merged_Huong_thom-1_GLCM_features.csv',
    r'D:\AIL301m\Feature extract\GLCM\Merged_Nep-87_GLCM_features.csv',
    r'D:\AIL301m\Feature extract\GLCM\Merged_Q-5_modify_GLCM_features.csv',
    r'D:\AIL301m\Feature extract\GLCM\Merged_Thien_uu-8_GLCM_features.csv',
    r'D:\AIL301m\Feature extract\GLCM\Merged_Xi-23_GLCM_features.csv'
]
results = []
accuracies = []
runtimes = []

In [3]:
for file_path in file_paths_GLCM:
    data = pd.read_csv(file_path)
    X = data.drop(['filename', 'label'], axis=1)
    y = data['label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
    
    
    # Define base models with specific parameters
    knn = KNeighborsClassifier(n_neighbors=4, weights='distance',metric='euclidean')
    ann = MLPClassifier(activation='tanh',alpha=0.05, hidden_layer_sizes=(50,) ,max_iter=1000, random_state=42)
    svm = SVC(kernel='rbf', C=100, gamma=0.01, random_state=42)
    rf = RandomForestClassifier(n_estimators=1200, max_features=3,criterion='entropy',max_depth=10, random_state=42)
    
    # Define meta-learner
    meta_learner = LogisticRegression(C=100, solver='liblinear',random_state=42)
    
    # Create the stacking classifier
    stacking_classifier = StackingClassifier(
        estimators=[('knn', knn), ('ann', ann), ('svm', svm), ('rf', rf)],
        final_estimator=meta_learner,
        cv=5
    )
    
    # Train the stacking classifier
    start_time = time.time()
    stacking_classifier.fit(X_train, y_train)
    y_pred = stacking_classifier.predict(X_test)
    end_time = time.time()
    runtime = end_time - start_time
    runtimes.append(runtime)
    
    # Calculate metrics
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"File: {file_path}")
    print("Stacking Model (KNN + ANN + SVM + RF):")
    print(f"Accuracy: {acc * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F-measure: {f1 * 100:.2f}%")
    print(f"Runtime: {runtime:.2f} seconds")  # Output runtime

    # Format the results to xx,xx%
    results.append([
        file_path,
        f"{acc * 100:.2f}".replace('.', ',') + '%',
        f"{precision * 100:.2f}".replace('.', ',') + '%',
        f"{recall * 100:.2f}".replace('.', ',') + '%',
        f"{f1 * 100:.2f}".replace('.', ',') + '%',
        f"{runtime:.2f} seconds"
    ])
    accuracies.append(acc)

# Specify the output file path
output_file = r'D:\AIL301m\time\Stacking\GLCM.csv'

# Write the results to a CSV file
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File', 'Accuracy', 'Precision', 'Recall', 'F-measure', 'Runtime (s)'])
    writer.writerows(results)
average_accuracy = np.mean(accuracies)
print(f"Overall Average Accuracy: {average_accuracy * 100:.2f}%") 

print(f"Results have been written to {output_file}")

File: D:\AIL301m\Feature extract\GLCM\Merged_BC-15_GLCM_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 72.16%
Precision: 72.47%
Recall: 72.16%
F-measure: 71.99%
Runtime: 76.98 seconds
File: D:\AIL301m\Feature extract\GLCM\Merged_Huong_thom-1_GLCM_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 79.71%
Precision: 79.70%
Recall: 79.71%
F-measure: 79.70%
Runtime: 81.79 seconds
File: D:\AIL301m\Feature extract\GLCM\Merged_Nep-87_GLCM_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 93.89%
Precision: 93.90%
Recall: 93.89%
F-measure: 93.89%
Runtime: 40.89 seconds


c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


File: D:\AIL301m\Feature extract\GLCM\Merged_Q-5_modify_GLCM_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 75.15%
Precision: 75.41%
Recall: 75.15%
F-measure: 75.05%
Runtime: 69.72 seconds
File: D:\AIL301m\Feature extract\GLCM\Merged_Thien_uu-8_GLCM_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 87.46%
Precision: 87.50%
Recall: 87.46%
F-measure: 87.46%
Runtime: 36.28 seconds


c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


File: D:\AIL301m\Feature extract\GLCM\Merged_Xi-23_GLCM_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 77.49%
Precision: 77.42%
Recall: 77.49%
F-measure: 77.40%
Runtime: 94.44 seconds
Overall Average Accuracy: 80.98%
Results have been written to D:\AIL301m\time\Stacking\GLCM.csv


In [4]:
file_paths_SIFTandBOW = [
    r'D:\AIL301m\Feature extract\SIFTandBOW\Merged_BC-15_sift_bow_features.csv',
    r'D:\AIL301m\Feature extract\SIFTandBOW\Merged_Huong_thom-1_sift_bow_features.csv',
    r'D:\AIL301m\Feature extract\SIFTandBOW\Merged_Nep-87_sift_bow_features.csv',
    r'D:\AIL301m\Feature extract\SIFTandBOW\Merged_Q-5_modify_sift_bow_features.csv',
    r'D:\AIL301m\Feature extract\SIFTandBOW\Merged_Thien_uu-8_sift_bow_features.csv',
    r'D:\AIL301m\Feature extract\SIFTandBOW\Merged_Xi-23_sift_bow_features.csv'
]
results = []
accuracies = []
runtimes = []

In [5]:
for file_path in file_paths_SIFTandBOW:
    data = pd.read_csv(file_path)
    X = data['features'].apply(eval).tolist()
    y = data['label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
    
    
    # Define base models with specific parameters
    knn = KNeighborsClassifier(n_neighbors=4, weights='distance',metric='euclidean')
    ann = MLPClassifier(activation='tanh',alpha=0.05, hidden_layer_sizes=(50,) ,max_iter=1000, random_state=42)
    svm = SVC(kernel='rbf', C=100, gamma=0.01, random_state=42)
    rf = RandomForestClassifier(n_estimators=1200, max_features=18,criterion='entropy',max_depth=10, random_state=42)
    
    # Define meta-learner
    meta_learner = LogisticRegression(C=100, solver='liblinear',random_state=42)
    
    # Create the stacking classifier
    stacking_classifier = StackingClassifier(
        estimators=[('knn', knn), ('ann', ann), ('svm', svm), ('rf', rf)],
        final_estimator=meta_learner,
        cv=5
    )
    
    # Train the stacking classifier
    start_time = time.time()
    stacking_classifier.fit(X_train, y_train)
    y_pred = stacking_classifier.predict(X_test)
    end_time = time.time()
    runtime = end_time - start_time
    runtimes.append(runtime)
    
    # Calculate metrics
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"File: {file_path}")
    print("Stacking Model (KNN + ANN + SVM + RF):")
    print(f"Accuracy: {acc * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F-measure: {f1 * 100:.2f}%")
    print(f"Runtime: {runtime:.2f} seconds")  # Output runtime

    # Format the results to xx,xx%
    results.append([
        file_path,
        f"{acc * 100:.2f}".replace('.', ',') + '%',
        f"{precision * 100:.2f}".replace('.', ',') + '%',
        f"{recall * 100:.2f}".replace('.', ',') + '%',
        f"{f1 * 100:.2f}".replace('.', ',') + '%',
        f"{runtime:.2f} seconds"
    ])
    accuracies.append(acc)

# Specify the output file path
output_file = r'D:\AIL301m\time\Stacking\SIFT.csv'

# Write the results to a CSV file
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File', 'Accuracy', 'Precision', 'Recall', 'F-measure', 'Runtime (s)'])
    writer.writerows(results)
average_accuracy = np.mean(accuracies)
print(f"Overall Average Accuracy: {average_accuracy * 100:.2f}%") 

print(f"Results have been written to {output_file}")

File: D:\AIL301m\Feature extract\SIFTandBOW\Merged_BC-15_sift_bow_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 78.42%
Precision: 78.59%
Recall: 78.42%
F-measure: 78.41%
Runtime: 56.38 seconds
File: D:\AIL301m\Feature extract\SIFTandBOW\Merged_Huong_thom-1_sift_bow_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 85.99%
Precision: 86.06%
Recall: 85.99%
F-measure: 85.99%
Runtime: 63.96 seconds
File: D:\AIL301m\Feature extract\SIFTandBOW\Merged_Nep-87_sift_bow_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 85.56%
Precision: 85.59%
Recall: 85.56%
F-measure: 85.56%
Runtime: 41.49 seconds
File: D:\AIL301m\Feature extract\SIFTandBOW\Merged_Q-5_modify_sift_bow_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 70.82%
Precision: 70.96%
Recall: 70.82%
F-measure: 70.81%
Runtime: 65.29 seconds
File: D:\AIL301m\Feature extract\SIFTandBOW\Merged_Thien_uu-8_sift_bow_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 79.61%
Precision: 

In [6]:
file_paths_GIST = [
    r'D:\AIL301m\Feature extract\GIST\Merged_BC-15_GIST_features.csv',
    r'D:\AIL301m\Feature extract\GIST\Merged_Huong_thom-1_GIST_features.csv',
    r'D:\AIL301m\Feature extract\GIST\Merged_Nep-87_GIST_features.csv',
    r'D:\AIL301m\Feature extract\GIST\Merged_Q-5_modify_GIST_features.csv',
    r'D:\AIL301m\Feature extract\GIST\Merged_Thien_uu-8_GIST_features.csv',
    r'D:\AIL301m\Feature extract\GIST\Merged_Xi-23_GIST_features.csv'
]
results = []
accuracies = []
runtimes = []

In [7]:
for file_path in file_paths_GIST:
    data = pd.read_csv(file_path)
    X = data.apply(lambda row: [float(x) for x in row['GistFeature'].split(',')], axis=1).tolist()
    y = data['label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
    
    
    # Define base models with specific parameters
    knn = KNeighborsClassifier(n_neighbors=4, weights='distance',metric='euclidean')
    ann = MLPClassifier(activation='tanh',alpha=0.05, hidden_layer_sizes=(50,) ,max_iter=1000, random_state=42)
    svm = SVC(kernel='rbf', C=100, gamma=0.01, random_state=42)
    rf = RandomForestClassifier(n_estimators=1200, max_features=23,criterion='entropy',max_depth=10, random_state=42)
    
    # Define meta-learner
    meta_learner = LogisticRegression(C=100, solver='liblinear',random_state=42)
    
    # Create the stacking classifier
    stacking_classifier = StackingClassifier(
        estimators=[('knn', knn), ('ann', ann), ('svm', svm), ('rf', rf)],
        final_estimator=meta_learner,
        cv=5
    )
    
    # Train the stacking classifier
    start_time = time.time()
    stacking_classifier.fit(X_train, y_train)
    y_pred = stacking_classifier.predict(X_test)
    end_time = time.time()
    runtime = end_time - start_time
    runtimes.append(runtime)
    
    # Calculate metrics
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"File: {file_path}")
    print("Stacking Model (KNN + ANN + SVM + RF):")
    print(f"Accuracy: {acc * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F-measure: {f1 * 100:.2f}%")
    print(f"Runtime: {runtime:.2f} seconds")  # Output runtime

    # Format the results to xx,xx%
    results.append([
        file_path,
        f"{acc * 100:.2f}".replace('.', ',') + '%',
        f"{precision * 100:.2f}".replace('.', ',') + '%',
        f"{recall * 100:.2f}".replace('.', ',') + '%',
        f"{f1 * 100:.2f}".replace('.', ',') + '%',
        f"{runtime:.2f} seconds"
    ])
    accuracies.append(acc)

# Specify the output file path
output_file = r'D:\AIL301m\time\Stacking\GIST.csv'

# Write the results to a CSV file
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File', 'Accuracy', 'Precision', 'Recall', 'F-measure', 'Runtime (s)'])
    writer.writerows(results)
average_accuracy = np.mean(accuracies)
print(f"Overall Average Accuracy: {average_accuracy * 100:.2f}%") 

print(f"Results have been written to {output_file}")

File: D:\AIL301m\Feature extract\GIST\Merged_BC-15_GIST_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 78.67%
Precision: 78.68%
Recall: 78.67%
F-measure: 78.67%
Runtime: 398.63 seconds
File: D:\AIL301m\Feature extract\GIST\Merged_Huong_thom-1_GIST_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 88.32%
Precision: 88.33%
Recall: 88.32%
F-measure: 88.32%
Runtime: 389.13 seconds
File: D:\AIL301m\Feature extract\GIST\Merged_Nep-87_GIST_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 91.25%
Precision: 91.27%
Recall: 91.25%
F-measure: 91.25%
Runtime: 259.84 seconds
File: D:\AIL301m\Feature extract\GIST\Merged_Q-5_modify_GIST_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 79.58%
Precision: 79.58%
Recall: 79.58%
F-measure: 79.58%
Runtime: 296.80 seconds
File: D:\AIL301m\Feature extract\GIST\Merged_Thien_uu-8_GIST_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 94.11%
Precision: 94.14%
Recall: 94.11%
F-measure: 94.11%
Runtim

In [8]:
file_paths_HOG = [
    r'D:\AIL301m\Feature extract\HOG\Merged_BC-15_HOG_features.csv',
    r'D:\AIL301m\Feature extract\HOG\Merged_Huong_thom-1_HOG_features.csv',
    r'D:\AIL301m\Feature extract\HOG\Merged_Nep-87_HOG_features.csv',
    r'D:\AIL301m\Feature extract\HOG\Merged_Q-5_modify_HOG_features.csv',
    r'D:\AIL301m\Feature extract\HOG\Merged_Thien_uu-8_HOG_features.csv',
    r'D:\AIL301m\Feature extract\HOG\Merged_Xi-23_HOG_features.csv'
]
results = []
accuracies = []
runtimes = []

In [3]:
for file_path in file_paths_HOG:
    data = pd.read_csv(file_path)
    X = data.apply(lambda row: [float(x) for x in row['HogFeature'].split(',')], axis=1).tolist()
    y = data['label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
    
    
    # Define base models with specific parameters
    knn = KNeighborsClassifier(n_neighbors=4, weights='distance',metric='euclidean')
    ann = MLPClassifier(activation='tanh',alpha=0.05, hidden_layer_sizes=(50,) ,max_iter=1000, random_state=42)
    svm = SVC(kernel='rbf', C=100, gamma=0.01, random_state=42)
    rf = RandomForestClassifier(n_estimators=1200, max_features=23,criterion='entropy',max_depth=10, random_state=42)
    
    # Define meta-learner
    meta_learner = LogisticRegression(C=100, solver='liblinear',random_state=42)
    
    # Create the stacking classifier
    stacking_classifier = StackingClassifier(
        estimators=[('knn', knn), ('ann', ann), ('svm', svm), ('rf', rf)],
        final_estimator=meta_learner,
        cv=5
    )
    
    # Train the stacking classifier
    start_time = time.time()
    stacking_classifier.fit(X_train, y_train)
    y_pred = stacking_classifier.predict(X_test)
    end_time = time.time()
    runtime = end_time - start_time
    runtimes.append(runtime)
    
    # Calculate metrics
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"File: {file_path}")
    print("Stacking Model (KNN + ANN + SVM + RF):")
    print(f"Accuracy: {acc * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F-measure: {f1 * 100:.2f}%")
    print(f"Runtime: {runtime:.2f} seconds")  # Output runtime

    # Format the results to xx,xx%
    results.append([
        file_path,
        f"{acc * 100:.2f}".replace('.', ',') + '%',
        f"{precision * 100:.2f}".replace('.', ',') + '%',
        f"{recall * 100:.2f}".replace('.', ',') + '%',
        f"{f1 * 100:.2f}".replace('.', ',') + '%',
        f"{runtime:.2f} seconds"
    ])
    accuracies.append(acc)

# Specify the output file path
output_file = r'D:\AIL301m\time\Stacking\HOG.csv'

# Write the results to a CSV file
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File', 'Accuracy', 'Precision', 'Recall', 'F-measure', 'Runtime (s)'])
    writer.writerows(results)
average_accuracy = np.mean(accuracies)
print(f"Overall Average Accuracy: {average_accuracy * 100:.2f}%") 

print(f"Results have been written to {output_file}")

NameError: name 'file_paths_HOG' is not defined

In [4]:
file_paths_basic = [
    r'D:\AIL301m\Feature extract\basic\18_feature\Merged_BC-15_basic_features.csv',
    r'D:\AIL301m\Feature extract\basic\18_feature\Merged_Huong_thom-1_basic_features.csv',
    r'D:\AIL301m\Feature extract\basic\18_feature\Merged_Nep-87_basic_features.csv',
    r'D:\AIL301m\Feature extract\basic\18_feature\Merged_Q-5_modify_basic_features.csv',
    r'D:\AIL301m\Feature extract\basic\18_feature\Merged_Thien_uu-8_basic_features.csv',
    r'D:\AIL301m\Feature extract\basic\18_feature\Merged_Xi-23_features.csv'
]
results = []
accuracies = []
runtimes = []

In [5]:
for file_path in file_paths_basic:
    data = pd.read_csv(file_path)
    X = data.drop(['filename', 'label'], axis=1)
    y = data['label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
    
    
    # Define base models with specific parameters
    knn = KNeighborsClassifier(n_neighbors=4, weights='distance',metric='euclidean')
    ann = MLPClassifier(activation='tanh',alpha=0.05, hidden_layer_sizes=(50,) ,max_iter=1000, random_state=42)
    svm = SVC(kernel='rbf', C=100, gamma=0.01, random_state=42)
    rf = RandomForestClassifier(n_estimators=1200, max_features=5,criterion='entropy',max_depth=10, random_state=42)
    
    # Define meta-learner
    meta_learner = LogisticRegression(C=100, solver='liblinear',random_state=42)
    
    # Create the stacking classifier
    stacking_classifier = StackingClassifier(
        estimators=[('knn', knn), ('ann', ann), ('svm', svm), ('rf', rf)],
        final_estimator=meta_learner,
        cv=5
    )
    
    # Train the stacking classifier
    start_time = time.time()
    stacking_classifier.fit(X_train, y_train)
    y_pred = stacking_classifier.predict(X_test)
    end_time = time.time()
    runtime = end_time - start_time
    runtimes.append(runtime)
    
    # Calculate metrics
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"File: {file_path}")
    print("Stacking Model (KNN + ANN + SVM + RF):")
    print(f"Accuracy: {acc * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F-measure: {f1 * 100:.2f}%")
    print(f"Runtime: {runtime:.2f} seconds")  # Output runtime

    # Format the results to xx,xx%
    results.append([
        file_path,
        f"{acc * 100:.2f}".replace('.', ',') + '%',
        f"{precision * 100:.2f}".replace('.', ',') + '%',
        f"{recall * 100:.2f}".replace('.', ',') + '%',
        f"{f1 * 100:.2f}".replace('.', ',') + '%',
        f"{runtime:.2f} seconds"
    ])
    accuracies.append(acc)

# Specify the output file path
output_file = r'D:\AIL301m\time\Stacking\basic_18.csv'

# Write the results to a CSV file
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File', 'Accuracy', 'Precision', 'Recall', 'F-measure', 'Runtime (s)'])
    writer.writerows(results)
average_accuracy = np.mean(accuracies)
print(f"Overall Average Accuracy: {average_accuracy * 100:.2f}%") 

print(f"Results have been written to {output_file}")

c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


File: D:\AIL301m\Feature extract\basic\18_feature\Merged_BC-15_basic_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 91.85%
Precision: 91.89%
Recall: 91.85%
F-measure: 91.84%
Runtime: 106.20 seconds
File: D:\AIL301m\Feature extract\basic\18_feature\Merged_Huong_thom-1_basic_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 94.31%
Precision: 94.37%
Recall: 94.31%
F-measure: 94.31%
Runtime: 97.19 seconds
File: D:\AIL301m\Feature extract\basic\18_feature\Merged_Nep-87_basic_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 97.68%
Precision: 97.70%
Recall: 97.68%
F-measure: 97.68%
Runtime: 47.04 seconds
File: D:\AIL301m\Feature extract\basic\18_feature\Merged_Q-5_modify_basic_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 95.17%
Precision: 95.21%
Recall: 95.17%
F-measure: 95.17%
Runtime: 70.51 seconds
File: D:\AIL301m\Feature extract\basic\18_feature\Merged_Thien_uu-8_basic_features.csv
Stacking Model (KNN + ANN + SVM + RF):
Accuracy: 97